https://github.com/fastai/courses/blob/master/deeplearning1/nbs/lesson4.ipynb

# Download & Unpack Data

In [1]:
%%bash
mkdir -p ./data/movielens
cd ./data/movielens
#wget http://files.grouplens.org/datasets/movielens/ml-latest.zip -O ml-latest.zip
wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2017-12-14 07:39:08--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 918269 (897K) [application/zip]
Saving to: 'ml-latest-small.zip'

     0K .......... .......... .......... .......... ..........  5%  157K 5s
    50K .......... .......... .......... .......... .......... 11%  330K 4s
   100K .......... .......... .......... .......... .......... 16% 4.66M 2s
   150K .......... .......... .......... .......... .......... 22% 18.2M 2s
   200K .......... .......... .......... .......... .......... 27%  314K 2s
   250K .......... .......... .......... .......... .......... 33% 17.5M 1s
   300K .......... .......... .......... .......... .......... 39% 11.2M 1s
   350K .......... .......... .......... .......... .......... 44% 12.4M 1s
   400K .......... ..

In [2]:
%%bash
cd ./data/movielens
#unzip -q ml-latest.zip
unzip -q ml-latest-small.zip

In [3]:
%%bash
ls ./data/movielens/ml-latest-small

README.txt
links.csv
movies.csv
ratings.csv
tags.csv


In [4]:
%%bash
head ./data/movielens/ml-latest-small/ratings.csv

userId,movieId,rating,timestamp
1,31,2.5,1260759144
1,1029,3.0,1260759179
1,1061,3.0,1260759182
1,1129,2.0,1260759185
1,1172,4.0,1260759205
1,1263,2.0,1260759151
1,1287,2.0,1260759187
1,1293,2.0,1260759148
1,1339,3.5,1260759125


# Import 

In [1]:
from theano.sandbox import cuda

import pandas as pd
import numpy as np

import utils; reload(utils)
from utils import *

from IPython.display import display

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 770 (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


In [2]:
ratings = pd.read_csv('./data/movielens/ml-latest-small/ratings.csv')
ratings.tail()

,userId,movieId,rating,timestamp
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663
100003,671,6565,3.5,1074784724


In [3]:
# Import movie file with movieId as index
movies = pd.read_csv('./data/movielens/ml-latest-small/movies.csv').set_index('movieId')
display(len(movies))
movies.head()

9125

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [4]:
# Lets create a mapping from id to title
nonContinuousMovieId2title = movies['title'].to_dict()

In [5]:
len(ratings)

100004

In [6]:
users = np.sort(ratings.userId.unique())
movies = np.sort(ratings.movieId.unique())
display(users)
display(movies)
len(movies)

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,
        19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,
        37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
        73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162,
       163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180,
       181, 182, 183, 184, 185

array([     1,      2,      3, ..., 162542, 162672, 163949])

9066

In [7]:
# Are the ids continuous integers?
def idContinuous(array):
    return np.array_equal(range(array.min(), array.max()+1), array)

print('Users: ' + str(idContinuous(users)))
print('Movies: ' + str(idContinuous(movies)))



Users: True
Movies: False


In [8]:
#Returns dictionary to map from non-continuous to continuous ID
def makeIdMapping (array):
    array = np.sort(array)
    mapping = {value:counter for counter, value in enumerate(array)}
    return mapping

# Get continuous mapping for data
movieNcid2Id = makeIdMapping(movies)
userMapping = makeIdMapping(users)

# Inplace replacement of mappings
ratings.tail()
ratings.movieId = ratings.movieId.apply(lambda x: movieNcid2Id[x])
ratings.userId = ratings.userId.apply(lambda x: userMapping[x])

ratings.tail()
# display(newMovies[:20])
# display(len(movies))
# display(len(newMovies))
# movies[newMovies[3]]

,userId,movieId,rating,timestamp
99999,670,4545,2.5,1065579370
100000,670,4546,4.0,1065149201
100001,670,4597,4.0,1070940363
100002,670,4610,2.5,1070979663
100003,670,4696,3.5,1074784724


In [9]:
#Create continuous movieID to title mapping

movieId2title = { cid:nonContinuousMovieId2title[ncid] for ncid, cid in movieNcid2Id.items()}


Create a training and validation set

In [10]:
# Mask is true if to be included in training set, otherwise validation
msk = np.random.rand(len(ratings)) < 0.8
# msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]
val.head()

,userId,movieId,rating,timestamp
2,0,859,3.0,1260759182
5,0,1017,2.0,1260759151
8,0,1083,3.5,1260759125
11,0,1140,1.0,1260759203
17,0,1962,2.5,1260759113


In [11]:
def crosstab(ratings, row = 'userId', column='movieId', value='rating'):
    return pd.crosstab(ratings[row], ratings[column], ratings[value], aggfunc=np.sum)

In [12]:
pd.crosstab(ratings['userId'], ratings['movieId'], ratings['rating'], aggfunc=np.sum)

movieId,0,1,2,3,4,5,6,7,8,9,...,9056,9057,9058,9059,9060,9061,9062,9063,9064,9065
userId,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create good sample for visualisation of results
Create a sample with active users and popular movies

In [13]:
"""
Returns the 
"""
def samplePopular(ratings, column, num = 10):
    g = ratings.groupby(column)['rating'].count()
    return g.sort_values(ascending=False)[1:num]

topUsers = samplePopular(ratings, 'userId')
topMovies = samplePopular(ratings, 'movieId')

display(topUsers)
display(topMovies)

userId
563    1868
623    1735
14     1700
72     1610
451    1340
467    1291
379    1063
310    1019
29     1011
Name: rating, dtype: int64

movieId
266     324
284     311
525     304
232     291
427     274
2062    259
0       247
472     244
522     237
Name: rating, dtype: int64

In [15]:
sample = ratings.join(topUsers, rsuffix='_r', how='inner', on='userId')
display(len(sample))

sample = sample.join(topMovies, rsuffix='_r', how='inner', on='movieId')
display(len(sample))

del sample['rating_r']

sample.head(20)

12637

75

,userId,movieId,rating,timestamp
962,14,0,2.0,997938310
5048,29,0,4.0,944943070
10214,72,0,5.0,1303464840
43329,310,0,3.0,898007830
51144,379,0,4.0,1048092869
61432,451,0,3.5,1133735252
65657,467,0,4.0,1296195523
82824,563,0,4.0,974712079
93534,623,0,5.0,1019126661
1032,14,232,5.0,997938437


# Build a model with embedding only and no bais term

In [17]:
num_factors = 50

# Input Node
user_in = Input(shape=(1,), dtype='int64', name='user_in')
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')

# Embedding Operation
u = Embedding(ratings.userId.nunique(), num_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
m = Embedding(ratings.movieId.nunique(), num_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

# Merge Operation
x = merge([u, m], mode='dot')
x = Flatten()(x)

# Build and compile model
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')


In [18]:
# Perform 1 iteration at low lr
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=1,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80167 samples, validate on 19837 samples
Epoch 1/1
80167/80167 [==============================] - 4s - loss: 9.9244 - val_loss: 3.8686


In [19]:
# Increase learning rate, and do 3 iterations
model.optimizer.lr = 0.01
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80167 samples, validate on 19837 samples
Epoch 1/3
80167/80167 [==============================] - 4s - loss: 3.1272 - val_loss: 2.0301
Epoch 2/3
80167/80167 [==============================] - 4s - loss: 2.3905 - val_loss: 1.6623
Epoch 3/3
80167/80167 [==============================] - 4s - loss: 2.2195 - val_loss: 1.5332


In [20]:
# Increase learning rate, and do 3 iterations
model.optimizer.lr = 0.001
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80167 samples, validate on 19837 samples
Epoch 1/3
80167/80167 [==============================] - 4s - loss: 2.1530 - val_loss: 1.4956
Epoch 2/3
80167/80167 [==============================] - 4s - loss: 2.1178 - val_loss: 1.4676
Epoch 3/3
80167/80167 [==============================] - 4s - loss: 2.0936 - val_loss: 1.4629


Lets see what the model is doing

In [252]:
sample.head()
sample['prediction'] = model.predict([sample.userId, sample.movieId])
sample['prediction'] = np.round(sample['prediction'], 1)

In [253]:
crosstab(sample)

movieId,0,232,266,284,427,472,522,525,2062
userId,,,,,,,,,
14,2.0,5.0,5.0,2.0,3.0,4.0,4.0,5.0,5.0
29,4.0,4.0,5.0,5.0,4.0,5.0,4.0,4.0,3.0
72,5.0,4.5,5.0,5.0,4.0,5.0,3.0,4.5,4.5
310,3.0,4.0,3.0,4.5,4.5,5.0,4.5,2.0,4.0
379,4.0,4.0,5.0,4.0,4.0,NaN,4.0,5.0,5.0
451,3.5,4.0,5.0,5.0,5.0,4.0,4.0,5.0,2.0
467,4.0,3.5,3.5,3.5,2.5,NaN,NaN,3.0,3.0
563,4.0,2.0,5.0,NaN,5.0,4.0,5.0,5.0,3.0
623,5.0,5.0,5.0,NaN,3.0,NaN,3.0,5.0,2.0


In [254]:
crosstab(sample, value='prediction')

movieId,0,232,266,284,427,472,522,525,2062
userId,,,,,,,,,
14,3.0,3.3,3.9,3.0,2.7,3.5,2.7,3.7,2.6
29,4.3,4.6,4.7,4.8,4.1,4.8,3.9,4.6,4.0
72,3.8,4.4,5.2,4.0,3.5,4.1,3.4,4.2,4.6
310,3.4,3.7,3.9,3.8,3.3,3.9,3.2,3.5,3.9
379,4.0,4.5,5.0,4.5,3.7,NaN,3.8,4.2,4.9
451,3.5,3.8,4.2,3.7,3.4,4.0,3.1,4.0,3.2
467,2.9,3.2,3.7,2.7,2.6,NaN,NaN,3.3,2.6
563,3.9,4.2,4.9,NaN,3.8,4.5,3.3,4.7,3.3
623,3.6,4.0,4.7,NaN,3.3,NaN,3.2,4.1,3.7


In [255]:
sample['diff'] = sample['prediction'] - sample['rating']

In [256]:
crosstab(sample, value='diff')

movieId,0,232,266,284,427,472,522,525,2062
userId,,,,,,,,,
14,1.0,-1.7,-1.1,1.0,-0.3,-0.5,-1.3,-1.3,-2.4
29,0.3,0.6,-0.3,-0.2,0.1,-0.2,-0.1,0.6,1.0
72,-1.2,-0.1,0.2,-1.0,-0.5,-0.9,0.4,-0.3,0.1
310,0.4,-0.3,0.9,-0.7,-1.2,-1.1,-1.3,1.5,-0.1
379,0.0,0.5,0.0,0.5,-0.3,NaN,-0.2,-0.8,-0.1
451,0.0,-0.2,-0.8,-1.3,-1.6,0.0,-0.9,-1.0,1.2
467,-1.1,-0.3,0.2,-0.8,0.1,NaN,NaN,0.3,-0.4
563,-0.1,2.2,-0.1,NaN,-1.2,0.5,-1.7,-0.3,0.3
623,-1.4,-1.0,-0.3,NaN,0.3,NaN,0.2,-0.9,1.7


# Build with Bias term

In [17]:
num_factors = 50
num_userId = ratings.userId.nunique()
num_movieId = ratings.movieId.nunique()
# Latent Factors
def opLatentFactors(inputName, n_in, n_out, reg=1e-4):
    inp = Input(shape=(1,), dtype='int64', name=inputName)
    emb = Embedding(n_in, n_out, input_length=1, W_regularizer=l2(1e-4))(inp)
    
    return (inp, emb)

(user_in, u) = opLatentFactors('user_in', num_userId, num_factors)

(movie_in, m) = opLatentFactors('movie_in', num_movieId, num_factors)

# Bais terms
def opBais(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

ub = opBais(user_in, num_userId)
mb = opBais(movie_in, num_movieId)

# Merge Operation
x = merge([u, m], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, mb], mode='sum')

# Build and compile model
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')


In [121]:
# Manual tweak parameters to train. 
# fast.ai achieved loss: 0.5747 - val_loss: 0.7979
# High batch size seems to work here
model.optimizer.lr = 0.0001
nb_epoch = 5


model.fit([trn.userId, trn.movieId], trn.rating, batch_size=256, nb_epoch=nb_epoch,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80167 samples, validate on 19837 samples
Epoch 1/5
80167/80167 [==============================] - 1s - loss: 0.5118 - val_loss: 0.8069
Epoch 2/5
80167/80167 [==============================] - 1s - loss: 0.5094 - val_loss: 0.8069
Epoch 3/5
80167/80167 [==============================] - 1s - loss: 0.5086 - val_loss: 0.8069
Epoch 4/5
80167/80167 [==============================] - 1s - loss: 0.5080 - val_loss: 0.8069
Epoch 5/5
80167/80167 [==============================] - 1s - loss: 0.5074 - val_loss: 0.8068


In [109]:
model.save_weights('./data/movielens/model/bias.h5')

In [16]:
model.load_weights('./data/movielens/model/bias.h5')

In [30]:
crosstab(sample)

movieId,0,232,266,284,427,472,522,525,2062
userId,,,,,,,,,
14,2.0,5.0,5.0,2.0,3.0,4.0,4.0,5.0,5.0
29,4.0,4.0,5.0,5.0,4.0,5.0,4.0,4.0,3.0
72,5.0,4.5,5.0,5.0,4.0,5.0,3.0,4.5,4.5
310,3.0,4.0,3.0,4.5,4.5,5.0,4.5,2.0,4.0
379,4.0,4.0,5.0,4.0,4.0,NaN,4.0,5.0,5.0
451,3.5,4.0,5.0,5.0,5.0,4.0,4.0,5.0,2.0
467,4.0,3.5,3.5,3.5,2.5,NaN,NaN,3.0,3.0
563,4.0,2.0,5.0,NaN,5.0,4.0,5.0,5.0,3.0
623,5.0,5.0,5.0,NaN,3.0,NaN,3.0,5.0,2.0


In [17]:
sample['prediction'] = model.predict([sample.userId, sample.movieId])
sample['prediction'] = np.round(sample['prediction'], 1)
crosstab(sample, value='prediction')

movieId,0,232,266,284,427,472,522,525,2062
userId,,,,,,,,,
14,1.5,4.8,4.3,3.0,3.2,4.9,3.9,4.9,5.0
29,4.3,3.9,4.9,4.7,3.8,5.5,3.7,4.3,3.4
72,3.5,4.6,5.0,5.0,3.9,4.6,3.2,4.8,4.4
310,3.0,4.0,2.9,4.5,4.3,4.4,4.3,2.3,4.1
379,3.9,3.9,5.3,4.0,2.6,NaN,4.2,4.8,5.0
451,3.5,3.9,5.0,4.8,4.2,4.3,3.6,4.9,2.4
467,3.7,3.4,3.6,3.3,2.5,NaN,NaN,3.1,3.1
563,3.8,2.2,4.9,NaN,4.8,4.0,5.0,6.3,2.7
623,5.0,5.1,5.3,NaN,3.0,NaN,3.2,4.8,2.5


In [56]:
# Lets predict values where we don't have a truth
predUserId = np.array([379, 467, 467, 563, 623, 623])
predMovieId = np.array([472, 472, 522, 284, 248, 472])
model.predict([predUserId, predMovieId])

array([[ 5.0564],
       [ 3.5002],
       [ 3.2185],
       [ 2.2246],
       [ 2.3228],
       [ 4.0009]], dtype=float32)

In [107]:
[(ind, movieId2title[ind]) for ind in sample['movieId'].unique() ]

[(0, 'Toy Story (1995)'),
 (232, "Kid in King Arthur's Court, A (1995)"),
 (266, 'Perez Family, The (1995)'),
 (284, 'Specialist, The (1994)'),
 (427, 'House of the Spirits, The (1993)'),
 (472, 'Renaissance Man (1994)'),
 (522, 'Wedding Gift, The (1994)'),
 (525, 'Little Rascals, The (1994)'),
 (2062, '\xc2\xa1Three Amigos! (1986)')]

# Result Visulisation

## Lets  look at the bias term
The bais term should indicate which movies are 'intrinstically' good or bad

In [18]:
# Lets just look at the movies with the top 2000 number of ratings
g=ratings.groupby('movieId')['rating'].count()
topMovies=g.sort_values(ascending=False)[:2000]
topMovies = np.array(topMovies.index)

In [233]:
get_movie_bias = Model(movie_in, mb)
topMovies
movie_bias = get_movie_bias.predict(topMovies)
movieRatings = [ (b[0], movieId, movieId2title[movieId]) for b, movieId in zip(movie_bias, topMovies)]
sorted(movieRatings, key=itemgetter(1))[1000:]

[(0.4306691, 2050, 'Forces of Nature (1999)'),
 (0.75869799, 2056, 'King and I, The (1956)'),
 (0.38707718, 2058, 'EDtv (1999)'),
 (1.3449683, 2062, 'Matrix, The (1999)'),
 (1.0501093, 2063, '10 Things I Hate About You (1999)'),
 (1.0565928, 2069, 'Go (1999)'),
 (0.55006558, 2070, 'Never Been Kissed (1999)'),
 (0.30987579, 2081, 'Pushing Tin (1999)'),
 (1.2528986, 2082, 'Election (1999)'),
 (0.57111019, 2083, 'eXistenZ (1999)'),
 (0.68932074, 2084, 'Entrapment (1999)'),
 (0.088376865, 2093, 'Dick Tracy (1990)'),
 (0.66435134, 2094, 'Mummy, The (1999)'),
 (0.8166244, 2098, "William Shakespeare's A Midsummer Night's Dream (1999)"),
 (0.64835161, 2103, 'Star Wars: Episode I - The Phantom Menace (1999)'),
 (0.77003431, 2114, 'Superman (1978)'),
 (0.59499753, 2115, 'Superman II (1980)'),
 (0.15500255, 2116, 'Superman III (1983)'),
 (-0.26989582, 2117, 'Superman IV: The Quest for Peace (1987)'),
 (1.4673057, 2120, 'Frankenstein (1931)'),
 (0.66954941, 2129, 'Rocky Horror Picture Show, The (1

In [234]:
# Worst 15 movies
sorted(movieRatings, key=itemgetter(0))[:15]

[(-0.89045441, 2869, 'Battlefield Earth (2000)'),
 (-0.6978718, 1893, 'Police Academy 6: City Under Siege (1989)'),
 (-0.48239586, 1127, 'Jaws 3-D (1983)'),
 (-0.44312072, 489, 'Super Mario Bros. (1993)'),
 (-0.41843429, 1242, 'Speed 2: Cruise Control (1997)'),
 (-0.30542514, 1890, 'Police Academy 3: Back in Training (1986)'),
 (-0.30008778, 1958, 'Howard the Duck (1986)'),
 (-0.29112419, 4608, '2 Fast 2 Furious (Fast and the Furious 2, The) (2003)'),
 (-0.28427437, 1384, 'Spice World (1997)'),
 (-0.27163914, 4666, "Charlie's Angels: Full Throttle (2003)"),
 (-0.26989582, 2117, 'Superman IV: The Quest for Peace (1987)'),
 (-0.23813057, 2176, 'Haunting, The (1999)'),
 (-0.23264232, 5628, 'Blade: Trinity (2004)'),
 (-0.19472913, 3059, 'Hollow Man (2000)'),
 (-0.19398305, 1453, 'Godzilla (1998)')]

In [237]:
# Top Movies
sorted(movieRatings, reverse=True)[:15]

[(1.9603179, 5328, 'Gladiator (1992)'),
 (1.7550346, 27, 'Persuasion (1995)'),
 (1.7534238, 284, 'Shawshank Redemption, The (1994)'),
 (1.6960554, 786, 'African Queen, The (1951)'),
 (1.6853266, 973, 'Ran (1985)'),
 (1.6789765, 862, 'Shall We Dance (1937)'),
 (1.6763607, 1510, 'Tom Jones (1963)'),
 (1.6677279, 3374, "Amores Perros (Love's a Bitch) (2000)"),
 (1.6478876,
  1034,
  'Raise the Red Lantern (Da hong deng long gao gao gua) (1991)'),
 (1.6433821, 931, 'Cinema Paradiso (Nuovo cinema Paradiso) (1989)'),
 (1.6208147, 695, 'Godfather, The (1972)'),
 (1.6153263, 1868, 'Central Station (Central do Brasil) (1998)'),
 (1.6136849, 746, 'All About Eve (1950)'),
 (1.6122957, 478, 'Shadowlands (1993)'),
 (1.6121042, 726, 'It Happened One Night (1934)')]

## Latent Factors

In [19]:
# Same thing as bias, use keras to extract embeddings
get_movie_emb = Model(movie_in, m)
movie_emb = np.squeeze(get_movie_emb.predict(topMovies))
movie_emb.shape
# Every movie as 50 embeddings.


(2000, 50)

Lets use PCA to reduce the number of dimensions


In [28]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
movie_pca_class = pca.fit(movie_emb.T)
movie_pca = movie_pca_class.components_
movie_pca.shape
# Now we only have 3 components per movie

(3, 2000)

In [29]:
movie_pca_class.explained_variance_ratio_

array([ 0.1105,  0.079 ,  0.05  ], dtype=float32)

In [30]:
movies_comp = [ [pca[0], pca[1], pca[2], movieId2title[movieId]] for pca, movieId in zip(movie_pca.T, topMovies)]
movies_comp

[[0.022285214, -0.061796281, 0.018012194, 'Forrest Gump (1994)'],
 [0.069639251, 0.023410851, -0.050290015, 'Pulp Fiction (1994)'],
 [0.06257502, -0.062079899, 0.001179551, 'Shawshank Redemption, The (1994)'],
 [0.074863851, -0.042585053, -0.074057586, 'Silence of the Lambs, The (1991)'],
 [0.067129426,
  -0.07833112,
  -0.0060474696,
  'Star Wars: Episode IV - A New Hope (1977)'],
 [0.032330848, -0.094506875, -0.019802321, 'Jurassic Park (1993)'],
 [0.039695289, -0.067727715, -0.078505151, 'Matrix, The (1999)'],
 [0.058946483, -0.071680479, 0.020160628, 'Toy Story (1995)'],
 [0.063372754, -0.055263199, 0.010528505, "Schindler's List (1993)"],
 [0.037674766,
  -0.053970497,
  -0.059905484,
  'Terminator 2: Judgment Day (1991)'],
 [0.074724004,
  -0.07593593,
  -0.055069316,
  'Star Wars: Episode V - The Empire Strikes Back (1980)'],
 [0.017455079, -0.084109172, -0.019859342, 'Braveheart (1995)'],
 [0.03414635, -0.064521708, -0.047502875, 'Back to the Future (1985)'],
 [0.076248147, 0.0

In [31]:
def explorePCA(movies_comp, compontent, n=10):
    print('Biggest Positive')
    display(sorted(movies_comp, key=itemgetter(compontent), reverse=True)[:n])
    print('Biggest Negavtive')
    display(sorted(movies_comp, key=itemgetter(compontent))[:n])

explorePCA(movies_comp, 2)

Biggest Positive


[[0.04309934, -0.050187498, 0.097401053, 'Beauty and the Beast (1991)'],
 [0.035168085, -0.0058007259, 0.081529282, 'Sling Blade (1996)'],
 [0.027109724, -0.059749972, 0.079631582, 'Aladdin (1992)'],
 [0.043996688, -0.027489843, 0.079014391, 'Babe (1995)'],
 [-0.0099550057, -0.021560399, 0.076308839, "You've Got Mail (1998)"],
 [0.030600963,
  -0.029684629,
  0.076079413,
  'Hunchback of Notre Dame, The (1996)'],
 [0.027630031, -0.008931159, 0.074957214, 'Christmas Story, A (1983)'],
 [0.032204397, -0.050676599, 0.069784433, 'Finding Nemo (2003)'],
 [0.031817421, 0.0323474, 0.068849452, 'Some Like It Hot (1959)'],
 [0.032604784, -0.038253833, 0.067053623, "Bug's Life, A (1998)"]]

Biggest Negavtive


[[0.048987258, -0.012728984, -0.10728132, 'Fight Club (1999)'],
 [0.06281957, -0.0378091, -0.087665007, 'Seven (a.k.a. Se7en) (1995)'],
 [0.034163594, -0.02541733, -0.085054658, 'Aliens (1986)'],
 [0.0046134167, 0.0095887939, -0.078751385, 'Natural Born Killers (1994)'],
 [0.031617198,
  -0.0057587381,
  -0.078725681,
  'L\xc3\xa9on: The Professional (a.k.a. The Professional) (L\xc3\xa9on) (1994)'],
 [0.039695289, -0.067727715, -0.078505151, 'Matrix, The (1999)'],
 [0.021561129, -0.026824858, -0.077959344, 'Total Recall (1990)'],
 [0.047776707, -0.015424082, -0.077025346, 'Office Space (1999)'],
 [0.074863851, -0.042585053, -0.074057586, 'Silence of the Lambs, The (1991)'],
 [0.028236767,
  -0.037494991,
  -0.067334123,
  'Austin Powers: International Man of Mystery (1997)']]

In [67]:
# Import plotly and enable offline mode
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [90]:
x = [i[0] for i in movies_comp[:]]
y = [i[1] for i in movies_comp[:]]
labels = [i[3] for i in movies_comp[:]]
labels

['Forrest Gump (1994)',
 'Pulp Fiction (1994)',
 'Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Jurassic Park (1993)',
 'Matrix, The (1999)',
 'Toy Story (1995)',
 "Schindler's List (1993)",
 'Terminator 2: Judgment Day (1991)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Braveheart (1995)',
 'Back to the Future (1985)',
 'Fargo (1996)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'American Beauty (1999)',
 'Independence Day (a.k.a. ID4) (1996)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Aladdin (1992)',
 'Fugitive, The (1993)',
 'Fight Club (1999)',
 'Dances with Wolves (1990)',
 'Seven (a.k.a. Se7en) (1995)',
 'Usual Suspects, The (1995)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Godfather, The (1972)',
 'Lion King, The (1994)',
 'Apollo 13 (1995)',
 'True Lies (1994)',
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
 'Ba

In [99]:
movie_trace = go.Scatter(
    x=x[0:50], 
    y=y[0:50], 
    mode='markers+text',
    marker=dict(
        size='8',
        color = np.random.randn(500), #set color equal to a variable
        colorscale='parula_rgb',
        showscale=False
    ),
    text=labels,
    textposition='right')

layout= go.Layout(
    title= 'First and Second PCA component on movies',
    hovermode= 'closest',
    xaxis= dict(
        title= 'PCA 1',
        zeroline= False,
    ),
    yaxis=dict(
        title= 'PCA 2',
    ),
    showlegend=False
)
fig= go.Figure(data=[movie_trace], layout=layout)
py.iplot(fig)

# Neural Net

In [25]:
(user_in, u) = opLatentFactors('user_in', num_userId, num_factors)

(movie_in, m) = opLatentFactors('movie_in', num_movieId, num_factors)

x = merge([u,m], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x) # Default assumes linear activation

nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')

In [35]:
# Manual tweak parameters to train. 
# fast.ai achieved loss: 0.5747 - val_loss: 0.7979
# High batch size seems to work here
model.optimizer.lr = 0.0003
nb_epoch = 8


model.fit([trn.userId, trn.movieId], trn.rating, batch_size=256, nb_epoch=nb_epoch,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 79876 samples, validate on 20128 samples
Epoch 1/8
79876/79876 [==============================] - 1s - loss: 0.5273 - val_loss: 0.8067
Epoch 2/8
79876/79876 [==============================] - 1s - loss: 0.5246 - val_loss: 0.8071
Epoch 3/8
79876/79876 [==============================] - 1s - loss: 0.5222 - val_loss: 0.8074
Epoch 4/8
79876/79876 [==============================] - 1s - loss: 0.5200 - val_loss: 0.8078
Epoch 5/8
79876/79876 [==============================] - 1s - loss: 0.5180 - val_loss: 0.8081
Epoch 6/8
79876/79876 [==============================] - 1s - loss: 0.5162 - val_loss: 0.8086ss
Epoch 7/8
79876/79876 [==============================] - 1s - loss: 0.5146 - val_loss: 0.8091
Epoch 8/8
79876/79876 [==============================] - 1s - loss: 0.5131 - val_loss: 0.8097
